In [1]:
import os
import re   # 정규표현식 활용
from dotenv import load_dotenv
import pandas as pd   # 데이터 처리
from langchain_community.document_loaders import TextLoader   # txt 파일 load
from langchain_community.document_loaders.csv_loader import CSVLoader # csv 파일 load
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate     # ChatPrompt template
from langchain_core.prompts import PromptTemplate         # Rag-Prompt template 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_core.runnables import RunnableParallel, RunnablePassthrough  # retriever data 병렬 전달
from langchain_core.output_parsers import StrOutputParser # chain output을 str로 받기 

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain.llms import WatsonxLLM

In [2]:
from langchain.schema import Document

def load_text_as_documents(file_path):
    file_path = file_path
    documents = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line_number, line in enumerate(file, 1):
            line = line.strip()  # 앞뒤 공백 제거
            if line:  # 빈 줄 무시
                doc = Document(
                    page_content=line,
                    metadata={"source": file_path, "line_number": line_number}
                )
                documents.append(doc)

    return documents

In [4]:
texts = load_text_as_documents("./data/saving_list.txt") + load_text_as_documents("./data/deposit_list.txt") + load_text_as_documents("./data/CMA_list.txt") + load_text_as_documents("./data/fund_list.txt")

총 61개의 문서로 로드되었습니다.
문서 1:
내용: kor_co_nm: 우리은행 | fin_prdt_nm: 우리SUPER주거래적금 | join_way: 영업점,인터넷,스마트폰,전화(텔레뱅킹) | mtrt_int: 만기 후 - 1개월이내 : 만기시점약정이율×50% - 1개월초과 6개월이내: 만기시점약정이율×30% - 6개월초과 : 만기시점약정이율×20%  ※ 만기시점 약정이율 : 일반정기적금 금리 | spcl_cnd: 1. 아래 각 항(가, 나)의 조건을 충족하는 경우 합산 최대 연 1.9%p 우대 가. 우리은행을 처음 거래하시는 고객 : 연 1.0%p 나. 거래실적 인정기간 동안 아래 거래실적을 계약기간별 필수기간(1년 : 6개월, 2년 : 12개월, 3년 : 18개월)이상 충족하는 경우 최대 연 0.9%p | join_deny: 1 | join_member: 실명의 개인 | max_limit: None
메타데이터: {'source': './data/saving_list.txt', 'line_number': 1}
--------------------------------------------------
문서 2:
내용: kor_co_nm: 우리은행 | fin_prdt_nm: WON적금 | join_way: 스마트폰,전화(텔레뱅킹) | mtrt_int: 만기 후 - 1개월이내 : 만기시점약정이율×50% - 1개월초과 6개월이내: 만기시점약정이율×30% - 6개월초과 : 만기시점약정이율×20%  ※ 만기시점 약정이율 : 일반정기적금 금리 | spcl_cnd: 1. 아래 각 항(가, 나)의 조건을 충족하는 경우 합산 최대 연 0.2%p 우대 가. 이 적금을 우리꿈통장, WON통장에 연결하여 가입하는 경우 : 0.1%p 나. 우리 오픈뱅킹 서비스에 타행계좌가 등록되어 있는 경우 : 연 0.1%p | join_deny: 1 | join_member: 실명의 개인 | max_limit: None
메타데이터: {'source': './data/saving_list.txt

In [5]:
len(texts)

270

In [7]:
#loader = TextLoader("./data/적금 리스트.txt", encoding='utf-8')
#documents = loader.load()
#text_splitter = RecursiveCharacterTextSplitter()
#texts = text_splitter.split_documents(documents)

In [8]:
#len(texts)

In [7]:
texts[120]

Document(metadata={'source': './data/CMA_list.txt', 'line_number': 21}, page_content='회사:KB증권, 이름:KB CMA 상품21, 운용방식:종금형, 수익률:2.5%, 이체 수수료 혜택:무료, CD/ATM기 수수료 혜택:무료, 이체 수수료 혜택 조건:CMA 주거래계좌 이용 시 온라인 이체 수수료 무료 혜택, CD/ATM기 수수료 혜택:CMA 주거래계좌 이용 시 전금융기관 CD, ATM기 이체출금 수수료 무료 혜택')

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",       # 한국어 특화 임베딩 모델
)

c:\Users\Jonnior_P\IBM_AI_Hackathon\gust-ibm-ai-hackathon\final_porj\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
DB_PATH = "./chroma_db"
docsearch = Chroma.from_documents(texts, embeddings,
                                  persist_directory=DB_PATH, 
                                  collection_name="investment_products_db")

In [12]:
load_dotenv()

True

In [13]:
# IBM Cloud 상의 watsonx SAS url은 provision된 region별로 정해져 있음. https://cloud.ibm.com/apidocs/watsonx-ai 참조.
project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": 'https://us-south.ml.cloud.ibm.com'
}

In [14]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 3000,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

model_id =  'mistralai/mistral-large' # 'meta-llama/llama-3-1-70b-instruct' # ModelTypes.LLAMA_2_70B_CHAT.value #
watsonx_llama2_korean = WatsonxLLM(
    model_id=model_id,
    url=wml_credentials.get("url"),
    apikey=wml_credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

C:\Users\Jonnior_P\AppData\Local\Temp\ipykernel_31572\2514157292.py:9: LangChainDeprecationWarning: The class `WatsonxLLM` was deprecated in LangChain 0.0.18 and will be removed in 0.3. An updated version of the class exists in the langchain-ibm package and should be used instead. To use it run `pip install -U langchain-ibm` and import as `from langchain_ibm import WatsonxLLM`.
  watsonx_llama2_korean = WatsonxLLM(


In [15]:
retriever = docsearch.as_retriever(search_kwargs={'k': 5})

In [19]:
# prompt 작성
prompt = PromptTemplate.from_template(
   """당신은 전문적인 금융상품 추천 가이드입니다. 사용자의 저축 계획과 투자 계획을 분석하여, 제공된 상품 정보(Context) 중에서 가장 적합한 상품들을 추천해야 합니다.

사용자 정보 및 계획:
{user_input}

고려사항:
1. 사용자의 나이
2. 저축 기간
3. 월 저축액
4. 목표 금리
5. 투자 성향 (원금 손실 가능성에 대한 태도)
6. 총 투자 가능 금액

추천 지침:
1. 사용자의 정보가 join_member에 맞지 않는 적금 상품을 추천하면 안됩니다.
2. 사용자의 저축 계획에 맞는 저축 상품을 추천하세요. 금리, 저축 기간, 월 저축액, max_limit을 고려하세요.
3. 사용자의 투자 계획에 맞는 투자 상품을 추천하세요. 수익률, 기간, 원금, 주식비중(%), 위험등급 을 고려하세요.
4. 추천 투자 상품에 대한 위험성에 따라 경고해주세요
5. 사용자의 나이에 맞는 우대 조건이 있는 적금 상품이면 좋아요
6. 추천한 상품이 진짜 최선의 선택인지 고려하세요.
7. 각 추천 상품에 대해 왜 이 상품이 사용자에게 적합한지 간단히 설명하세요.
8. 우대 조건에 대해 간단히 설명하세요.
9. 추천된 상품들의 예상 수익을 계산하여 제시하세요.
10. 사용자의 계획과 추천 상품 사이에 차이가 있다면 이를 얘기하고 조언을 제공하세요.

Context (사용 가능한 금융 상품 정보):
{context}

추천 및 분석:
"""
)

# chain 생성
rag_chain = (
    {"context": retriever, "user_input": RunnablePassthrough()}
    | prompt
    | watsonx_llama2_korean
    | StrOutputParser()
)

In [20]:
user_input = '''안녕하세요, 30세의 사용자님 
1. **저축 계획**:
   - 매월 저축액: 월 저축 및 투자 가용금액 150만원의 70%인 105만원을 이자 3.5-4% 적금에 가입합니다.
   - 5년후 저축액: 105만원 X 60개월 + 적금이자 = 6300만원 +적금이자

2. **투자 계획**:
   - 초기 투자금: 원금 손실의 가능성이 없기 때문에 원금 모두
   - 투자 방법: 정기예금 및 CMA혼합 포트폴리오를 구성합니다. 정기예금 50%, CMA 50%로 분산 투자합니다.
   - 예상 수익률: 연평균 3.5% 가정
     - 근거: CMA평균 수익률 3.5%, 정기예금 최고 금리 평균 연 3.76%
   - 5년 후 예상 총 수익금(원금+수익): 원금 1000만원 + 수익(5 X 1000만원 X 0.035=175만원) = 약 1175만원
'''
response = rag_chain.invoke(user_input)

In [21]:
print(response)


### 저축 계획 추천 상품

1. **아이엠뱅크 - 내손안에 적금**
   - **금리**: 최고우대금리 연 0.55%
   - **저축 기간**: 사용자의 저축 기간에 맞춤
   - **월 저축액**: 105만원 (사용자의 저축 계획에 맞춤)
   - **max_limit**: 100만원 (사용자의 저축 계획에 맞춤)
   - **우대 조건**:
     - 인터넷/모바일뱅킹을 통한 최초 가입: 연 0.20%p
     - 최근 3개월 이내 인터넷/모바일뱅킹을 통한 이체거래 3회 이상: 연 0.10%p
     - 수수료우대 통장 보유 고객: 연 0.20%p
     - 모바일뱅킹을 통한 가입: 연 0.05%p

   **왜 이 상품이 적합한가?**
   - 사용자의 저축 계획에 맞는 금리와 저축 기간을 제공합니다.
   - 다양한 우대 조건을 통해 최고우대금리를 받을 수 있습니다.

   **예상 수익**:
   - 5년 후 예상 수익: 105만원 X 60개월 + 적금이자 = 6300만원 + 적금이자
   - 적금이자: 6300만원 X 0.0055 = 346,500원
   - 총 예상 수익: 6300만원 + 346,500원 = 6334만 6,500원

### 투자 계획 추천 상품

1. **아이엠뱅크 - iM함께예금**
   - **금리**: 최고우대금리 연 0.45%
   - **투자 기간**: 5년
   - **투자 금액**: 500만원 (정기예금 50%)
   - **우대 조건**:
     - 전월 총 수신 평잔실적 또는 상품 가입 전 첫만남플러스 통장 보유: 연 0.10%p
     - 당행 주택청약상품 보유: 연 0.10%p
     - 신규일 "iM함께적금" 동시 가입 및 만기 보유: 연 0.10%p
     - 당행 오픈뱅킹서비스에 다른 은행 계좌 등록: 연 0.10%p
     - 인터넷/모바일뱅킹을 통한 가입: 연 0.05%p

   **왜 이 상품이 적합한가?**
   - 사용자의 투자 계획에 맞는 금리와 투자 기간을 제공합니다.
 

In [33]:
# retriever 테스트
retriever.get_relevant_documents(user_input)

C:\Users\Jonnior_P\AppData\Local\Temp\ipykernel_4916\3075586040.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents(user_input)


[Document(metadata={'line_number': 4, 'source': './data/적금 리스트.txt'}, page_content='kor_co_nm: 아이엠뱅크 | fin_prdt_nm: 내손안에 적금 | join_way: 스마트폰 | mtrt_int: 만기 후 1개월 미만 경과: 약정이자율 x 50% 만기 후 3개월 미만 경과: 약정이자율 x 25% 만기 후 3개월 이상 경과: 약정이자율 x 10% | spcl_cnd: *최고우대금리 : 연0.55%p -당행 인터넷/모바일뱅킹을 통하여 최초로 적립식예금 가입 시 : 연0.20%p -상품 가입 전 최근 3개월 이내 당행 인터넷/모바일뱅킹을 통한 이체거래 3회 이상 : 연0.10%p -상품 가입 전 수수료우대 통장 보유 고객 : 연0.20%p * 해당 상품을 모바일뱅킹을 통해 가입 : 연0.05%p | join_deny: 1 | join_member: 실명의 개인 | max_limit: 1000000'),
 Document(metadata={'line_number': 36, 'source': './data/적금 리스트.txt'}, page_content='kor_co_nm: 국민은행 | fin_prdt_nm: KB 특★한 적금 | join_way: 스마트폰 | mtrt_int: - 1개월 이내 : 기본이율 X 50% - 1개월 초과  ~ 3개월 이내 : 기본이율 X 30% - 3개월 초과 : 0.1% | spcl_cnd: 항목별 적용 조건 충족시, 최고 연 4.0%p ① 목표달성 축하 우대이율: 최고 연 1.0%p     50만원 이하: 연 0.5%p, 50만원 초과: 연 1.0%p  ② 별 모으기 우대이율 : 최고 연 1.0%p     10개: 연 0.5%p, 20개: 연 1.0%p ③ 함께해요 우대이율: 최고 연 2.0%p | join_deny: 1 | join_member: 실명의 개인 | max_limit: None'),
 Document(metadata={'line_number': 2